In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import time
import itertools
from ISLP.models import (ModelSpec as MS,
                         summarize, poly)
from astropy.io import ascii
from sklearn.model_selection import train_test_split

In [13]:
import os
PathToRepo = os.path.normpath(os.getcwd() + os.sep + os.pardir)

FullData = ascii.read(PathToRepo + '\\Data\\m12i_res7100_mhdcv.disk.ecsv', guess = False)
df = FullData.to_pandas()


In [14]:
df["IsMigratorInt"] = abs(df['Rcyl_form'] - df['Rcyl'] > 1.5).astype(int)
DataFinal = df.drop([
        'rsph', 'x', 'y', 'vx', 'vy', 'rsph_form', 'x_form', 
        'y_form', 'z_form', 'vx_form', 'vy_form',
        'vz_form', 'Rcyl_form', 'phi_form', 'vRcyl_form', 'vphi_form'
       ], axis = 1)
x = DataFinal.drop(['IsMigratorInt'], axis = 1)
y = DataFinal['IsMigratorInt']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [15]:
x_train

,z,vz,Rcyl,phi,vRcyl,vphi,age,mass,feh,oh,ch,mgh,ofe,cfe,mgfe
2041578,-0.284,305.711,0.555,97.661,-54.217,77.534,9.323,5911.742,-0.472,0.003,-0.223,-0.159,0.475,0.249,0.312
4144270,-0.262,-94.536,0.933,358.080,-30.892,385.319,4.261,7819.511,0.052,0.495,0.385,0.281,0.444,0.333,0.229
4515910,-0.096,168.146,1.267,63.090,-227.944,112.316,5.845,5774.697,-0.168,0.279,0.127,0.098,0.447,0.296,0.267
5642865,-0.028,-17.259,8.383,97.800,-38.099,186.265,0.595,7209.740,0.109,0.579,0.482,0.335,0.469,0.372,0.226
1237188,0.388,-119.339,0.300,199.641,-63.093,-25.536,8.247,5575.687,-0.483,-0.076,-0.117,-0.229,0.407,0.365,0.254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249467,-0.352,-110.264,0.939,77.200,-120.636,104.535,6.334,9964.321,-0.155,0.287,0.144,0.109,0.441,0.298,0.263
5157699,-0.159,-91.188,2.322,252.176,216.676,231.529,6.062,4673.067,-0.321,0.123,-0.045,-0.051,0.444,0.277,0.271
2215104,-0.230,1.363,0.693,291.096,187.758,-171.618,8.613,6673.675,-0.458,-0.031,-0.184,-0.190,0.427,0.274,0.268
1484405,-0.197,-26.438,0.441,114.174,57.351,-229.632,8.568,5107.385,-0.428,0.051,-0.157,-0.112,0.479,0.271,0.316


In [16]:
X = MS(x_train).fit_transform(x_train)
model = sm.GLM(
    y_train
    , X
    , family = sm.families.Binomial()
    )
regr = model.fit()

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
summary = regr.summary()
# Display and interpret results
with open(PathToRepo + '\\Tables\\GLMalldata.tex', 'w') as f:
    f.write(summary.as_latex())
summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:          IsMigratorInt   No. Observations:              4673756
Model:                            GLM   Df Residuals:                  4673740
Model Family:                Binomial   Df Model:                           15
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.2262e+06
Date:                Thu, 12 Dec 2024   Deviance:                   2.4525e+06
Time:                        20:48:25   Pearson chi2:                 1.31e+12
No. Iterations:                     7   Pseudo R-squ. (CS):             0.1437
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      1.3763      0.059     23.430      0.000       1.261       1.491
z             -0.0073      0.006     -1.156      0.248      -0.020       0.005
vz         -4.841e-06   1.67e-05     -0.289      0.773   -3.77e-05     2.8e-05
Rcyl           0.0890      0.001    103.240      0.000       0.087       0.091
phi            0.0008    1.6e-05     50.857      0.000       0.001       0.001
vRcyl      -4.811e-05   1.39e-05     -3.459      0.001   -7.54e-05   -2.08e-05
vphi           0.0074   1.89e-05    390.139      0.000       0.007       0.007
age           -0.8559      0.003   -296.956      0.000      -0.862      -0.850
mass        1.478e-05   1.04e-06     14.166      0.000    1.27e-05    1.68e-05
feh            2.2578      4.455      0.507      0.612      -6.474      10.989
oh           -12.7902      3.641     -3.513      0.000     -19.926      -5.655
ch            -3.1128      3.638     -0.856      0.392     -10.244       4.018
mgh           10.2918      3.642      2.826      0.005       3.154      17.430
ofe           -5.9680      3.641     -1.639      0.101     -13.103       1.167
cfe           -9.2799      3.639     -2.550      0.011     -16.411      -2.148
mgfe          30.5741      3.642      8.394      0.000      23.435      37.713
==============================================================================
"""

In [19]:
accuracy_score(y_test, regr.predict(MS(x_test).fit_transform(x_test)) > 0.5)

0.9021892437780288

Using only the predictors chosen in variable selection

In [20]:
x_train = x_train.drop(["cfe", "feh", "mass", "ofe", "oh", "phi", "vRcyl", "vphi", "vz", "z", "mgfe"], axis = 1)
x_test = x_test.drop(["cfe", "feh", "mass", "ofe", "oh", "phi", "vRcyl", "vphi", "vz", "z", "mgfe"], axis = 1)

In [21]:
X = MS(x_train).fit_transform(x_train)
model = sm.GLM(
    y_train
    , X
    , family = sm.families.Binomial()
    )
regr = model.fit()

accuracy_score(y_test, regr.predict(MS(x_test).fit_transform(x_test)) > 0.5)

0.8910504604429839

In [22]:
summary = regr.summary()
# Display and interpret results
with open(PathToRepo + '\\Tables\\GLMalldataBestSubsetPred.tex', 'w') as f:
    f.write(summary.as_latex())
summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:          IsMigratorInt   No. Observations:              4673756
Model:                            GLM   Df Residuals:                  4673751
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.3370e+06
Date:                Thu, 12 Dec 2024   Deviance:                   2.6740e+06
Time:                        20:48:33   Pearson chi2:                 5.53e+12
No. Iterations:                     7   Pseudo R-squ. (CS):             0.1021
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      3.4941      0.015    232.927      0.000       3.465       3.524
Rcyl           0.0591      0.001     74.136      0.000       0.058       0.061
age           -0.9033      0.002   -420.958      0.000      -0.908      -0.899
ch           -23.9466      0.064   -373.191      0.000     -24.072     -23.821
mgh           21.7211      0.068    321.292      0.000      21.589      21.854
==============================================================================
"""